In [1]:
import ultralytics
ultralytics.checks()
from ultralytics import YOLO
from PIL import Image

import torch
import yaml

import os
import random
from ultralytics.utils.plotting import plot_labels
import numpy as np
import pandas as pd
import sys

module_path = os.path.abspath(os.path.join('..'))
print(module_path)
module_path = module_path+'/data_preprocessing'
print(module_path)

if module_path not in sys.path:
    sys.path.append(module_path)

import visualization_utils as visutils

Ultralytics YOLOv8.0.199 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (NVIDIA RTX A5000, 24248MiB)
Setup complete ✅ (32 CPUs, 251.6 GB RAM, 0.7/125.8 GB disk)


/vast/palmer/home.grace/eec42/BirdDetector/src
/vast/palmer/home.grace/eec42/BirdDetector/src/data_preprocessing


In [2]:
device = "0" if torch.cuda.is_available() else "cpu"
if device == "0":
    torch.cuda.set_device(0) # Set to your desired GPU number

print(device)

0


In [3]:
model_name = 'pfeifer_penguins_poland_yolov8m_120epoch'
model = YOLO('runs/detect/' + model_name + '/weights/best.pt')

IOU_THRESHOLD = 0.1
CONF_THRESHOLD = 0.2

In [33]:
fname = "data.yaml"
stream = open(fname, 'r')
data = yaml.safe_load(stream)
img_path = data['path'] + '/test/'
img_path

selected_img = random.choices(os.listdir(img_path + '/images/'), k=1)
selected_img


['global_birds_pfeifer_Fregata_Island_2016_Chinstrap_penguins_205_patch_0.0_0.0_448_448.jpg']

In [34]:
# Predictions with right iou & confidence threshold

results = model.predict(
        #model = 'runs/detect/pfeifer_yolov8n_70epoch_default_batch32_dropout0.3',
        source = [os.path.join(img_path + 'images/', img) for img in selected_img],
        conf = CONF_THRESHOLD, 
        iou = IOU_THRESHOLD,
        show=False,
        save=False
    )

results


0: 640x640 19 birds, 6.0ms
Speed: 18.0ms preprocess, 6.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'bird'}
 orig_img: array([[[67, 69, 69],
         [66, 68, 68],
         [66, 68, 68],
         ...,
         [87, 92, 91],
         [84, 89, 87],
         [73, 78, 76]],
 
        [[66, 68, 68],
         [65, 67, 67],
         [66, 68, 68],
         ...,
         [77, 82, 81],
         [79, 84, 82],
         [74, 79, 77]],
 
        [[68, 70, 70],
         [68, 70, 70],
         [70, 72, 72],
         ...,
         [67, 72, 71],
         [71, 76, 74],
         [72, 77, 75]],
 
        ...,
 
        [[66, 68, 68],
         [62, 64, 64],
         [57, 59, 59],
         ...,
         [47, 52, 53],
         [43, 48, 49],
         [40, 45, 46]],
 
        [[67, 69, 69],
         [64, 66, 66],
         [59, 61, 61],
         ...,
         [50, 55, 56],
         [47, 52, 53],
         [44, 49, 50]],
 
        [[65, 67, 67],
         [63, 65,

In [35]:
len(results)

1

### Visualize

In [36]:
result = results[0]
img = selected_img[0]

detection_boxes = []
save_path = '/vast/palmer/home.grace/eec42/BirdDetector/src/model/runs/detect/' + model_name + '/prediction_' + os.path.basename(result.path).split('.jpg')[0] + '.jpg'
for detect in range(len(result.boxes.cls)):
    det = {}
    det['conf'] = result.boxes.conf[detect].cpu()
    det['category'] = result.boxes.cls[detect].cpu()
    coords = result.boxes.xywhn[detect].cpu()
    det['bbox'] = [coords[0]-coords[2]/2, coords[1]-coords[3]/2, coords[2], coords[3]]
    detection_boxes.append(det)

print("Number of detections: ", len(detection_boxes))

im_path = os.path.join(img_path + 'images/', img)
visutils.draw_bounding_boxes_on_file(im_path, save_path, detection_boxes,
                                confidence_threshold=0.0, detector_label_map=None,
                                thickness=1,expansion=0, colormap=['Red'])

selected_label = img_path + 'labels/' + os.path.basename(result.path).split('.jpg')[0] + '.txt'
detection_boxes = []
true_bboxes = torch.tensor([], dtype=torch.float32)
true_classes = torch.tensor([], dtype=torch.float32)
df = pd.read_csv(selected_label, sep='\t', header=None, index_col=False)
for irow, row in df.iterrows():  
    det = {}
    det['conf'] = None
    det['category'] = row[0]
    true_classes = torch.cat((true_classes, torch.tensor([row[0]], dtype=torch.float32)), 0)
    det['bbox'] = [row[1]-row[3]/2, row[2]-row[4]/2, row[3], row[4]]
    true_bboxes = torch.cat((true_bboxes, torch.tensor([[row[1]-row[3]/2, row[2]-row[4]/2, row[1]+row[3]/2, row[2]+row[4]/2]], dtype=torch.float32)), 0)

    detection_boxes.append(det)

# Draw annotations
print("Number of detection labels: ", len(detection_boxes))
save_path2 = '/vast/palmer/home.grace/eec42/BirdDetector/src/model/runs/detect/' + model_name + '/EVALUATOR_TEST_prediction_label_' + os.path.basename(result.path).split('.hpg')[0] + '.jpg'
visutils.draw_bounding_boxes_on_file(save_path, save_path2, detection_boxes,
                                confidence_threshold=0.0, detector_label_map=None,
                                thickness=1,expansion=0, colormap=['SpringGreen'])


Number of detections:  19
Number of detection labels:  17


### Match detections with ground truths

In [37]:
def box_iou(box1, box2, eps=1e-7):
    """
    Calculate intersection-over-union (IoU) of boxes. Both sets of boxes are expected to be in (x1, y1, x2, y2) format.
    Based on https://github.com/pytorch/vision/blob/master/torchvision/ops/boxes.py

    Args:
        box1 (torch.Tensor): A tensor of shape (N, 4) representing N bounding boxes.
        box2 (torch.Tensor): A tensor of shape (M, 4) representing M bounding boxes.
        eps (float, optional): A small value to avoid division by zero. Defaults to 1e-7.

    Returns:
        (torch.Tensor): An NxM tensor containing the pairwise IoU values for every element in box1 and box2.
    """

    # inter(N,M) = (rb(N,M,2) - lt(N,M,2)).clamp(0).prod(2)
    (a1, a2), (b1, b2) = box1.unsqueeze(1).chunk(2, 2), box2.unsqueeze(0).chunk(2, 2)
    inter = (torch.min(a2, b2) - torch.max(a1, b1)).clamp_(0).prod(2)

    # IoU = inter / (area1 + area2 - inter)
    return inter / ((a2 - a1).prod(2) + (b2 - b1).prod(2) - inter + eps)


In [38]:
pred_classes = result.boxes.cls
pred_classes = pred_classes.cpu()
pred_bboxes = result.boxes.xyxyn
pred_bboxes = pred_bboxes.cpu()

In [39]:
pred_bboxes

tensor([[7.3661e-01, 1.5245e-01, 7.9937e-01, 2.1814e-01],
        [7.7830e-01, 3.1854e-02, 8.4100e-01, 9.7088e-02],
        [0.0000e+00, 7.1415e-02, 5.6110e-02, 1.3782e-01],
        [4.6395e-01, 3.2652e-03, 5.2825e-01, 6.9059e-02],
        [2.6355e-01, 0.0000e+00, 3.3269e-01, 3.1120e-02],
        [6.9119e-01, 1.2916e-03, 7.5584e-01, 6.7873e-02],
        [6.7284e-01, 1.4715e-01, 7.3572e-01, 2.1079e-01],
        [2.2937e-01, 1.6163e-01, 2.9279e-01, 2.2360e-01],
        [6.8913e-01, 8.1183e-02, 7.5224e-01, 1.4439e-01],
        [5.4456e-01, 3.5860e-02, 6.0829e-01, 1.0114e-01],
        [8.1954e-01, 1.0968e-01, 8.8081e-01, 1.7149e-01],
        [5.9723e-01, 1.1590e-04, 6.6117e-01, 3.7605e-02],
        [5.8292e-02, 9.2950e-02, 1.2168e-01, 1.5734e-01],
        [2.8510e-02, 0.0000e+00, 9.8326e-02, 5.1345e-02],
        [1.0550e-01, 1.9162e-02, 1.6885e-01, 8.5249e-02],
        [9.2208e-01, 3.3882e-01, 9.8723e-01, 4.0320e-01],
        [8.4570e-01, 4.1182e-02, 9.0683e-01, 1.0141e-01],
        [3.609

In [40]:
true_bboxes

tensor([[ 0.0000e+00,  8.2589e-02,  5.5804e-02,  1.3839e-01],
        [ 2.9018e-02, -4.8572e-17,  8.4821e-02,  5.1339e-02],
        [ 1.0938e-01,  2.9018e-02,  1.6518e-01,  8.4821e-02],
        [ 6.4732e-02,  8.9286e-02,  1.2054e-01,  1.4509e-01],
        [ 4.6652e-01,  1.7857e-02,  5.2232e-01,  7.3661e-02],
        [ 5.6027e-01,  5.3571e-02,  6.1607e-01,  1.0938e-01],
        [ 6.9643e-01,  8.9286e-03,  7.5223e-01,  6.4732e-02],
        [ 6.8750e-01,  8.4821e-02,  7.4330e-01,  1.4062e-01],
        [ 6.7634e-01,  1.4955e-01,  7.3214e-01,  2.0536e-01],
        [ 7.4107e-01,  1.5625e-01,  7.9688e-01,  2.1205e-01],
        [ 7.6562e-01,  1.2277e-01,  8.2143e-01,  1.7857e-01],
        [ 7.8125e-01,  2.9018e-02,  8.3705e-01,  8.4821e-02],
        [ 8.3929e-01,  4.0179e-02,  8.9509e-01,  9.5982e-02],
        [ 8.2366e-01,  1.1161e-01,  8.7946e-01,  1.6741e-01],
        [ 8.4598e-01,  2.2098e-01,  9.0179e-01,  2.7679e-01],
        [ 9.3304e-01,  3.4375e-01,  9.8884e-01,  3.9955e-01],
        

In [41]:
iou = box_iou(true_bboxes, pred_bboxes)

In [42]:
iou

tensor([[0.0000, 0.0000, 0.8201, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.7992, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.7437, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.6788, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.6368, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.4578, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0

In [14]:
#pred_bboxes = result.boxes
print(pred_bboxes.xyxy[0])
print(pred_bboxes.xywh[0])

print(pred_bboxes.xywh[0][0] - pred_bboxes.xywh[0][2]/2)
print(pred_bboxes.xyxy[0][0])
print(pred_bboxes.xywh[0][0] - pred_bboxes.xywh[0][2]/2 == pred_bboxes.xyxy[0][0])
print(pred_bboxes.xywh[0][0] + pred_bboxes.xywh[0][2]/2 == pred_bboxes.xyxy[0][2])
print(pred_bboxes.xywh[0][1] - pred_bboxes.xywh[0][3]/2)
print(pred_bboxes.xyxy[0][3])
print(pred_bboxes.xywh[0][1] - pred_bboxes.xywh[0][3]/2 == pred_bboxes.xyxy[0][1])
print(pred_bboxes.xywh[0][1] + pred_bboxes.xywh[0][3]/2 == pred_bboxes.xyxy[0][3])


AttributeError: 'Tensor' object has no attribute 'xyxy'

In [43]:
# match_predictions method from YOLOv8 code - try to reuse it to simplify

def match_predictions(pred_classes, true_classes, iou, use_scipy=False):
    """
    Matches predictions to ground truth objects (pred_classes, true_classes) using IoU.

    Args:
        pred_classes (torch.Tensor): Predicted class indices of shape(N,).
        true_classes (torch.Tensor): Target class indices of shape(M,).
        iou (torch.Tensor): An NxM tensor containing the pairwise IoU values for predictions and ground of truth
        use_scipy (bool): Whether to use scipy for matching (more precise).

    Returns:
        (torch.Tensor): Correct tensor of shape(N,1) for 1 IoU thresholds.
    """
    # Dx10 matrix, where D - detections, 10 - IoU thresholds
    correct = np.zeros((pred_classes.shape[0], 1)).astype(bool)
    # LxD matrix where L - labels (rows), D - detections (columns)
    correct_class = true_classes[:, None] == pred_classes
    iou = iou * correct_class  # zero out the wrong classes
    iou = iou.cpu().numpy()
    threshold = IOU_THRESHOLD
    #for i, threshold in enumerate(self.iouv.cpu().tolist()):
    if use_scipy:
        # WARNING: known issue that reduces mAP in https://github.com/ultralytics/ultralytics/pull/4708
        import scipy  # scope import to avoid importing for all commands
        cost_matrix = iou * (iou >= threshold)
        if cost_matrix.any():
            labels_idx, detections_idx = scipy.optimize.linear_sum_assignment(cost_matrix, maximize=True)
            valid = cost_matrix[labels_idx, detections_idx] > 0
            if valid.any():
                correct[detections_idx[valid]] = True
    else:
        matches = np.nonzero(iou >= threshold)  # IoU > threshold and classes match
        matches = np.array(matches).T
        if matches.shape[0]:
            if matches.shape[0] > 1:
                matches = matches[iou[matches[:, 0], matches[:, 1]].argsort()[::-1]]
                matches = matches[np.unique(matches[:, 1], return_index=True)[1]]
                # matches = matches[matches[:, 2].argsort()[::-1]]
                matches = matches[np.unique(matches[:, 0], return_index=True)[1]]
            correct[matches[:, 1].astype(int)] = True
    return torch.tensor(correct, dtype=torch.bool, device=pred_classes.device)


In [44]:
correct = match_predictions(pred_classes, true_classes, iou)  # what they call tp in the code !!!!

In [45]:
correct

tensor([[ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True]])

### Compute metrics

In [46]:
TP = correct.sum()
FN = len(true_bboxes) - TP
FP = len(pred_bboxes) - TP
print(TP, FN, FP)

tensor(15) tensor(2) tensor(4)


In [ ]:
# Print recall curve


In [ ]:
# Print precision curve

In [ ]:
def ap_per_class(tp,
                 conf,
                 pred_cls,
                 target_cls,
                 plot=False,
                 on_plot=None,
                 save_dir=Path(),
                 names=(),
                 eps=1e-16,
                 prefix=''):
    """
    Computes the average precision per class for object detection evaluation.

    Args:
        tp (np.ndarray): Binary array indicating whether the detection is correct (True) or not (False).
        conf (np.ndarray): Array of confidence scores of the detections.
        pred_cls (np.ndarray): Array of predicted classes of the detections.
        target_cls (np.ndarray): Array of true classes of the detections.
        plot (bool, optional): Whether to plot PR curves or not. Defaults to False.
        on_plot (func, optional): A callback to pass plots path and data when they are rendered. Defaults to None.
        save_dir (Path, optional): Directory to save the PR curves. Defaults to an empty path.
        names (tuple, optional): Tuple of class names to plot PR curves. Defaults to an empty tuple.
        eps (float, optional): A small value to avoid division by zero. Defaults to 1e-16.
        prefix (str, optional): A prefix string for saving the plot files. Defaults to an empty string.

    Returns:
        (tuple): A tuple of six arrays and one array of unique classes, where:
            tp (np.ndarray): True positive counts at threshold given by max F1 metric for each class.Shape: (nc,).
            fp (np.ndarray): False positive counts at threshold given by max F1 metric for each class. Shape: (nc,).
            p (np.ndarray): Precision values at threshold given by max F1 metric for each class. Shape: (nc,).
            r (np.ndarray): Recall values at threshold given by max F1 metric for each class. Shape: (nc,).
            f1 (np.ndarray): F1-score values at threshold given by max F1 metric for each class. Shape: (nc,).
            ap (np.ndarray): Average precision for each class at different IoU thresholds. Shape: (nc, 10).
            unique_classes (np.ndarray): An array of unique classes that have data. Shape: (nc,).
            p_curve (np.ndarray): Precision curves for each class. Shape: (nc, 1000).
            r_curve (np.ndarray): Recall curves for each class. Shape: (nc, 1000).
            f1_curve (np.ndarray): F1-score curves for each class. Shape: (nc, 1000).
            x (np.ndarray): X-axis values for the curves. Shape: (1000,).
            prec_values: Precision values at mAP@0.5 for each class. Shape: (nc, 1000).
    """

    # Sort by objectness
    i = np.argsort(-conf)
    tp, conf, pred_cls = tp[i], conf[i], pred_cls[i]

    # Find unique classes
    unique_classes, nt = np.unique(target_cls, return_counts=True)
    nc = unique_classes.shape[0]  # number of classes, number of detections

    # Create Precision-Recall curve and compute AP for each class
    x, prec_values = np.linspace(0, 1, 1000), []

    # Average precision, precision and recall curves
    ap, p_curve, r_curve = np.zeros((nc, tp.shape[1])), np.zeros((nc, 1000)), np.zeros((nc, 1000))
    for ci, c in enumerate(unique_classes):
        i = pred_cls == c
        n_l = nt[ci]  # number of labels
        n_p = i.sum()  # number of predictions
        if n_p == 0 or n_l == 0:
            continue

        # Accumulate FPs and TPs
        fpc = (1 - tp[i]).cumsum(0)
        tpc = tp[i].cumsum(0)

        # Recall
        recall = tpc / (n_l + eps)  # recall curve
        r_curve[ci] = np.interp(-x, -conf[i], recall[:, 0], left=0)  # negative x, xp because xp decreases

        # Precision
        precision = tpc / (tpc + fpc)  # precision curve
        p_curve[ci] = np.interp(-x, -conf[i], precision[:, 0], left=1)  # p at pr_score

        # AP from recall-precision curve
        for j in range(tp.shape[1]):
            ap[ci, j], mpre, mrec = compute_ap(recall[:, j], precision[:, j])
            if plot and j == 0:
                prec_values.append(np.interp(x, mrec, mpre))  # precision at mAP@0.5

    prec_values = np.array(prec_values)  # (nc, 1000)

    # Compute F1 (harmonic mean of precision and recall)
    f1_curve = 2 * p_curve * r_curve / (p_curve + r_curve + eps)
    names = [v for k, v in names.items() if k in unique_classes]  # list: only classes that have data
    names = dict(enumerate(names))  # to dict
    if plot:
        plot_pr_curve(x, prec_values, ap, save_dir / f'{prefix}PR_curve.png', names, on_plot=on_plot)
        #plot_mc_curve(x, f1_curve, save_dir / f'{prefix}F1_curve.png', names, ylabel='F1', on_plot=on_plot)
        #plot_mc_curve(x, p_curve, save_dir / f'{prefix}P_curve.png', names, ylabel='Precision', on_plot=on_plot)
        #plot_mc_curve(x, r_curve, save_dir / f'{prefix}R_curve.png', names, ylabel='Recall', on_plot=on_plot)

    i = smooth(f1_curve.mean(0), 0.1).argmax()  # max F1 index
    p, r, f1 = p_curve[:, i], r_curve[:, i], f1_curve[:, i]  # max-F1 precision, recall, F1 values
    tp = (r * nt).round()  # true positives
    fp = (tp / (p + eps) - tp).round()  # false positives
    return tp, fp, p, r, f1, ap, unique_classes.astype(int), p_curve, r_curve, f1_curve, x, prec_values



In [18]:
def compute_ap(recall, precision):
    """
    Compute the average precision (AP) given the recall and precision curves.

    Args:
        recall (list): The recall curve.
        precision (list): The precision curve.

    Returns:
        (float): Average precision.
        (np.ndarray): Precision envelope curve.
        (np.ndarray): Modified recall curve with sentinel values added at the beginning and end.
    """

    # Append sentinel values to beginning and end
    mrec = np.concatenate(([0.0], recall, [1.0]))
    mpre = np.concatenate(([1.0], precision, [0.0]))

    # Compute the precision envelope
    mpre = np.flip(np.maximum.accumulate(np.flip(mpre)))

    # Integrate area under curve
    method = 'interp'  # methods: 'continuous', 'interp'
    if method == 'interp':
        x = np.linspace(0, 1, 101)  # 101-point interp (COCO)
        ap = np.trapz(np.interp(x, mrec, mpre), x)  # integrate
    else:  # 'continuous'
        i = np.where(mrec[1:] != mrec[:-1])[0]  # points where x-axis (recall) changes
        ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])  # area under curve

    return ap, mpre, mrec